# Cover Metrics

In [24]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import rioxarray


data_dir = Path("../data")
plots_dir = data_dir / "outputs" / "plots"
metrics_dir = plots_dir / "metrics"
net_cdf_dir = metrics_dir / "x1-y1-z1" / "net_cdf"
cover_metrics_dir = metrics_dir / "cover_metrics"

cover_metrics_dir.mkdir(parents=True, exist_ok=True)

In [25]:
cover_limits = pd.read_csv(data_dir / "outputs/lidar_assessed_storey_limits.csv")
cover_limits = cover_limits.drop(columns=['confidence', 'comments'])
cover_limits = cover_limits.dropna(subset=['la_ground_limit'])
cover_limits

,site_plot_id,veg_ground_total,veg_understorey_total,veg_midstorey_total,veg_upperstorey_total,veg_total_cover_total,la_ground_limit,la_understorey_limit,la_midstorey_limit
0,AGG_O_01_P1,9.0,3.0,NaN,20.0,25.0,1.0,4.0,NaN
1,AGG_O_01_P2,6.0,3.0,NaN,9.0,15.0,1.0,4.0,NaN
2,AGG_O_01_P3,3.0,3.0,NaN,17.0,23.0,1.0,4.0,NaN
3,AGG_O_01_P4,9.0,18.0,19.0,2.0,42.0,1.0,10.0,30.0
4,AGG_O_01_P5,12.0,10.0,17.0,2.0,32.0,1.0,10.0,30.0
...,...,...,...,...,...,...,...,...,...
325,ULY_Y_96_P1,31.0,20.0,7.0,0.0,38.0,1.0,3.0,17.0
326,ULY_Y_96_P2,35.0,39.0,7.0,0.0,50.0,1.0,3.0,17.0
327,ULY_Y_96_P3,29.0,26.0,3.0,3.0,40.0,1.0,3.0,17.0
328,ULY_Y_96_P4,26.0,35.0,2.0,0.0,49.0,1.0,3.0,17.0


In [26]:
def get_cover_dataarray(ds: xr.Dataset, lower: int, upper: int | None):
    ds_slice = ds.sel(z=slice(lower, upper))

    inside_weight = ds_slice["vox_inside_weight"].sum(dim='z')
    enter_weight = ds_slice["vox_enter_weight"].max(dim="z")

    cover_weight = (inside_weight / enter_weight) * 100

    return cover_weight

def get_cover_dataset(
    site_plot_id: str, understorey_limit: int | None, midstorey_limit: int | None
):
    ds = xr.open_dataset(net_cdf_dir / f"{site_plot_id}.nc", decode_coords='all') 
    ds.load()

    ground_lower = 1
    ground_upper = 1

    ground_cover = get_cover_dataarray(ds, 1, 1)

    understorey_lower = ground_upper + 1
    understorey_upper = (
        understorey_limit if not pd.isna(understorey_limit) else (understorey_lower - 1)
    )

    midstorey_lower = understorey_upper + 1
    midstorey_upper = (
        midstorey_limit if not pd.isna(midstorey_limit) else (midstorey_lower - 1)
    )

    upperstorey_lower = midstorey_upper + 1
    upperstorey_upper = None


    if understorey_upper >= understorey_lower:
        understorey_cover = get_cover_dataarray(ds, understorey_lower, understorey_upper)
    else:
        understorey_cover = xr.DataArray(np.nan, dims=['y', 'x'], coords={'y': ground_cover.y, 'x': ground_cover.x})

    if midstorey_upper >= midstorey_lower:
        midstorey_cover = get_cover_dataarray(ds, midstorey_lower, midstorey_upper)
    else:
        midstorey_cover = xr.DataArray(np.nan, dims=['y', 'x'], coords={'y': ground_cover.y, 'x': ground_cover.x})

    # There's always an upperstorey
    if upperstorey_lower <= ds.z.max().item():
        upperstorey_cover = get_cover_dataarray(ds, upperstorey_lower, upperstorey_upper)
    else:
        upperstorey_cover = xr.DataArray(np.nan, dims=['y', 'x'], coords={'y': ground_cover.y, 'x': ground_cover.x})

    ds.close()

    cover_ds = xr.Dataset(
        {
            "ground_lcapture": ground_cover,
            "understorey_lcapture": understorey_cover,
            "midstorey_lcapture": midstorey_cover,
            "upperstorey_lcapture": upperstorey_cover,
        }
    )

    cover_ds.attrs['site_plot_id'] = site_plot_id
    cover_ds.attrs['ground_limits'] = str((ground_lower, ground_upper))
    cover_ds.attrs['understorey_limits'] = str((understorey_lower, understorey_upper))
    cover_ds.attrs['midstorey_limits'] = str((midstorey_lower, midstorey_upper))
    cover_ds.attrs['upperstorey_limits'] = str((upperstorey_lower, upperstorey_upper))

    return cover_ds


In [ ]:
def get_cover_metrics(row: pd.Series):
    site_plot_id = row.name
    understorey_limit = row['la_understorey_limit']
    midstorey_limit = row['la_midstorey_limit']

    cover_ds = get_cover_dataset(site_plot_id, understorey_limit, midstorey_limit)
    # cover_ds.to_netcdf(cover_metrics_dir / f"{site_plot_id}.nc")
    cover_ds.rio.to_raster(cover_metrics_dir / f"{site_plot_id}.tif")

    return pd.concat([row, cover_ds.mean().to_pandas()])

In [28]:
cover_metrics = cover_limits.set_index('site_plot_id').apply(get_cover_metrics, axis=1)

AttributeError: 'Dataset' object has no attribute 'sd'

In [21]:
cover_metrics[['ground_lcapture', 'understorey_lcapture', 'midstorey_lcapture', 'upperstorey_lcapture']] = cover_metrics[['ground_lcapture', 'understorey_lcapture', 'midstorey_lcapture', 'upperstorey_lcapture']].fillna(0)
cover_metrics[['veg_ground_total', 'veg_understorey_total', 'veg_midstorey_total', 'veg_upperstorey_total', 'veg_total_cover_total']] = cover_metrics[['veg_ground_total', 'veg_understorey_total', 'veg_midstorey_total', 'veg_upperstorey_total', 'veg_total_cover_total']].fillna(0)
cover_metrics

,veg_ground_total,veg_understorey_total,veg_midstorey_total,veg_upperstorey_total,veg_total_cover_total,la_ground_limit,la_understorey_limit,la_midstorey_limit,ground_lcapture,understorey_lcapture,midstorey_lcapture,upperstorey_lcapture
site_plot_id,,,,,,,,,,,,
AGG_O_01_P1,9.0,3.0,0.0,20.0,25.0,1.0,4.0,NaN,47.601193,13.235867,0.000000,48.252944
AGG_O_01_P2,6.0,3.0,0.0,9.0,15.0,1.0,4.0,NaN,40.263256,14.210006,0.000000,49.759959
AGG_O_01_P3,3.0,3.0,0.0,17.0,23.0,1.0,4.0,NaN,47.357775,18.726996,0.000000,53.492825
AGG_O_01_P4,9.0,18.0,19.0,2.0,42.0,1.0,10.0,30.0,62.733297,60.169925,29.405346,16.072162
AGG_O_01_P5,12.0,10.0,17.0,2.0,32.0,1.0,10.0,30.0,47.549081,53.888520,47.322764,20.408356
...,...,...,...,...,...,...,...,...,...,...,...,...
ULY_Y_96_P1,31.0,20.0,7.0,0.0,38.0,1.0,3.0,17.0,62.903931,18.124774,39.186705,0.914479
ULY_Y_96_P2,35.0,39.0,7.0,0.0,50.0,1.0,3.0,17.0,75.263325,17.993120,17.045596,3.416992
ULY_Y_96_P3,29.0,26.0,3.0,3.0,40.0,1.0,3.0,17.0,79.879841,26.677435,12.287468,0.495731


In [22]:
cover_metrics.to_csv(cover_metrics_dir / "cover_metrics.csv")